In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [2]:
df = pd.read_csv("/content/c_train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cluster
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
 12  Cluster      891 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB


In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Cluster'],
      dtype='object')

Selecting Features

In [5]:
X = df.loc[:,['Pclass','Sex', 'Age', 'SibSp','Parch','Fare','Embarked']].values
y = df.loc[:,'Survived'].values

Encoding Categorical Variables

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,1] = le.fit_transform(X[:,1])

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[-1])],remainder='passthrough')
X = ct.fit_transform(X)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

Tuning ANN

In [9]:
import tensorflow as tf
ann = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units = 9, activation='relu'),
    tf.keras.layers.Dense(units = 20, activation='relu'),
    tf.keras.layers.Dropout(rate = 0.1),
    tf.keras.layers.Dense(units = 1, activation='sigmoid')
])

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 666)

In [11]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        0.01,
        decay_steps=60,
        decay_rate=0.90,
        staircase=True)
opt = tf.keras.optimizers.Adam(lr_schedule)
ann.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size=64, epochs = 60)
ann.summary()

Epoch 1/60
10/10 [==============================] - 1s 2ms/step - loss: 0.6585 - accuracy: 0.6019
Epoch 2/60
10/10 [==============================] - 0s 2ms/step - loss: 0.5174 - accuracy: 0.7689
Epoch 3/60
10/10 [==============================] - 0s 2ms/step - loss: 0.4748 - accuracy: 0.7689
Epoch 4/60
10/10 [==============================] - 0s 2ms/step - loss: 0.4533 - accuracy: 0.7833
Epoch 5/60
10/10 [==============================] - 0s 2ms/step - loss: 0.4432 - accuracy: 0.7929
Epoch 6/60
10/10 [==============================] - 0s 2ms/step - loss: 0.4299 - accuracy: 0.7961
Epoch 7/60
10/10 [==============================] - 0s 3ms/step - loss: 0.4337 - accuracy: 0.7913
Epoch 8/60
10/10 [==============================] - 0s 2ms/step - loss: 0.4164 - accuracy: 0.8234
Epoch 9/60
10/10 [==============================] - 0s 2ms/step - loss: 0.4137 - accuracy: 0.8138
Epoch 10/60
10/10 [==============================] - 0s 4ms/step - loss: 0.4004 - accuracy: 0.8299
Epoch 11/60
10/10 [

In [12]:
res = ann.evaluate(X_test, y_test)[1]

9/9 [==============================] - 0s 2ms/step - loss: 0.4556 - accuracy: 0.8209


In [13]:
test = pd.read_csv('/content/c_test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cluster
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0


In [14]:
test = pd.read_csv('/content/c_test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cluster
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0


In [15]:
X_test = test.loc[:,['Pclass','Sex', 'Age', 'SibSp','Parch','Fare','Embarked']].values

In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_test[:,1] = le.fit_transform(X_test[:,1])

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[-1])],remainder='passthrough')
X_test = ct.fit_transform(X_test)

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_test = sc.fit_transform(X_test)

In [19]:
y_pred = ann.predict(X_test)
y_pred

array([[7.47541189e-02],
       [1.86370939e-01],
       [3.47411036e-02],
       [8.64458680e-02],
       [2.53177404e-01],
       [6.65085316e-02],
       [7.26158738e-01],
       [1.22951746e-01],
       [5.94042838e-01],
       [3.42954695e-02],
       [8.17652941e-02],
       [2.30630606e-01],
       [9.97888446e-01],
       [2.64732242e-02],
       [9.81304526e-01],
       [9.62355793e-01],
       [5.84363043e-02],
       [9.57481563e-02],
       [1.62462920e-01],
       [4.26609367e-01],
       [2.92491913e-01],
       [9.60748911e-01],
       [9.98534322e-01],
       [5.29068768e-01],
       [9.63936925e-01],
       [3.13568413e-02],
       [9.99646664e-01],
       [8.16407800e-02],
       [2.52785593e-01],
       [2.78717399e-01],
       [4.22334969e-02],
       [4.63265181e-02],
       [2.20656961e-01],
       [2.55325735e-01],
       [5.68737328e-01],
       [1.18948817e-01],
       [3.78243685e-01],
       [4.69620824e-01],
       [8.13715756e-02],
       [4.08615798e-01],


In [20]:
y_pred[y_pred>=0.5] = 1
y_pred[y_pred<0.5] = 0

In [21]:
y_pred = [int(i) for i in y_pred]
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,


In [22]:
output_2 = test.iloc[:,0:1]
output_2['Survived'] = y_pred
output_2

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [23]:
output_2.to_csv('Tuned_Ann.csv',index=False)